In [1]:
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
from typing import Union
import json
from datetime import datetime
load_dotenv()

True

# Metadata

The dataset contains the following information:

Country

Year

Spill_ID = A unique code which will enable each individual spill to be individually identified

FlightType = The type of flight the detection was made during: National = "N", CEPCO = "C", Super CEPCO = "SC", Tour d’Horizon = “TDH”

Date = The date of the detection (dd.mm.yyyy)

Time_UTC = The time of the detection in UTC (hh:mm)

Wind_speed = The wind speed at the time of the detection (m/s)

Wind_direc = The wind direction in degrees at the time of the detection (degrees)

Latitude = The latitude of the detection (decimal degrees, WGS84)

Longitude = The longitude of the detection (decimal degrees, WGS84)

Length__km = The length of the detection (km)

Width__km = The width of the detection (km)

Area__km2_ = The area of the detection (km2)

Spill_cat = Spill/pollution category: Mineral Oil = “Oil", Other Substance = "Other substance" , "Unknown substance" = “Unknown”

EstimVol_m = If Spill_cat="Oil", then estimated min. volume of oil spill. Volume of the detection confirmed/observed as mineral oil as calculated using the Bonn
Agreement Oil Appearance Code using the lower figure (BAOAC minimum) in m3.

Vol_Category = Category of the detection: <0,1m3 = “1”, <0,1-1m3 = “2”, 1-10 m3 = “3”, 10-100 m3 = “4”, >100 m3 = “5”

Type_substance = If Spill_cat="Other substance" or "Unknown. Product name or type of OS or GAR substances that could be identified (in case of known
polluter, or via visual identification - cf. BAOAC Atlas). - Examples for OS: vegetable oils (palm oil sun flower oil, soya oil etc.), fish oil, molasses, various chemicals (methanol, biodiesels/FAME, toluene, paraffines etc.); Examples of GAR: solid cargo residues (e.g. coal residues), plastics, fish nets, …
OR "Unknown" (in case the type of substance could not be identified)

Polluter = Type of polluter source: Offshore Installation = “Rig”, Vessel = “Ship”, Other Polluter or source (e.g. land based source) = “Other”, Unknown = “Unknwon” (in case of an “orphan” spill that cannot be linked to a polluter)

Remarks = Any additional information to inform on particular situations
Description of marine litter sightings

# Convert JSON to CSV

In [2]:
def read_json_data_from_dir(dir_path: Union[str, Path]) -> pd.DataFrame:
    if isinstance(dir_path, str):
        dir_path = Path(dir_path)
    data_bucket = []
    for file_path in dir_path.glob("*.json"):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            json_data = {**json_data['properties'], **json_data['geometry']}
            json_data = {key: [value] for key, value in json_data.items()}
        df = pd.DataFrame(json_data)
        data_bucket.append(df)
    data_bucket = [df.dropna(axis=1, how='all') for df in data_bucket]
    data_df = pd.concat(data_bucket, ignore_index=True)
    return data_df

In [3]:
data = read_json_data_from_dir(os.getenv("DATA_PATH"))
data.head()

,OBJECTID,HELCOM_ID,Country,Year,Spill_ID,Latitude,Longitude,Length__km,Width__km_,Area__km2_,...,Polluter,Date,Time_UTC,x,y,Wind_speed,Wind_direc,FlightType,Remarks,Type_Substance
0,1003,4560,Sweden,2015,SE-83,63.519167,19.789833,1.70,0.09,0.153,...,Ship,1446768000000,-2209117500000,2.202994e+06,9.228699e+06,NaN,NaN,NaN,NaN,NaN
1,1004,4559,Sweden,2015,SE-60,57.733333,11.666667,5.50,0.20,1.100,...,Unknown,1438905600000,-2209122000000,1.298727e+06,7.911507e+06,NaN,NaN,NaN,NaN,NaN
2,1005,4558,Sweden,2015,SE-81,58.616660,17.250000,0.70,0.30,0.210,...,Unknown,1446163200000,-2209106100000,1.920261e+06,8.097990e+06,NaN,NaN,NaN,NaN,NaN
3,1006,4557,Sweden,2015,DK-23,56.183000,12.493500,1.10,0.10,0.110,...,Unknown,1432857600000,-2209111680000,1.390770e+06,7.594932e+06,20,223.0,NaN,NaN,NaN
4,1007,4556,Sweden,2015,FI/SE-3,59.925000,19.427780,69.68,15.25,1.179,...,Ship,1432598400000,-2209116600000,2.162691e+06,8.383059e+06,NaN,NaN,NaN,NaN,NaN


In [11]:
data = data[data["Year"] > 2015]
data.sort_values("Area__km2_", ascending=False)

,OBJECTID,HELCOM_ID,Country,Year,Spill_ID,Latitude,Longitude,Length__km,Width__km_,Area__km2_,...,Time_UTC,x,y,Wind_speed,Wind_direc,FlightType,Remarks,Type_Substance,Date_standard,Datetime_standard
275,6053,5789,Sweden,2023,23UTSLÄPP0179,56.093056,14.786111,NaN,NaN,45.175,...,-2209108260000,1.645982e+06,7.576963e+06,0,NaN,N,Marco Polo ran aground three times. Major oil ...,NaN,2023-10-22,2023-10-22 02:00:00
147,376,5187,Sweden,2019,SE-38,55.714500,15.335167,9.500,2.700,25.650,...,-2209120800000,1.707103e+06,7.501789e+06,N/A,NaN,N,NaN,NaN,2019-07-12,2019-07-12 02:00:00
122,267,5296,Germany,2020,GE/20UTSLÄPP0077,54.809833,13.981333,3.880,4.800,18.624,...,-2209116480000,1.556395e+06,7.325046e+06,NaN,NaN,N,Detected by Sweden (20UTSLÄPP0077),NaN,2020-05-23,2020-05-23 02:00:00
135,286,5277,Estonia,2020,EE-07,59.391600,23.908700,7.781,3.121,14.387,...,-2209136460000,2.661504e+06,8.265512e+06,2.05,200.0,N,EMSA detection confirmed,NaN,2020-09-25,2020-09-25 02:00:00
304,705,4858,Sweden,2017,SE-60,55.283333,12.800000,14.000,1.000,14.000,...,-2209130400000,1.424889e+06,7.417051e+06,NaN,NaN,N,Daylight,NaN,2017-07-31,2017-07-31 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,883,4680,Finland,2016,FI-2,60.010500,22.860500,NaN,NaN,NaN,...,-2209122000000,2.544819e+06,8.402076e+06,NaN,NaN,N,OIL ON ICE,NaN,2016-01-30,2016-01-30 01:00:00
363,886,4677,Finland,2016,FI-12,61.126233,21.438350,1.000,2.000,NaN,...,-2209117200000,2.386506e+06,8.654866e+06,NaN,NaN,N,Amount so small that it could not be measured,NaN,2016-05-14,2016-05-14 02:00:00
364,887,4676,Finland,2016,FI-11,61.114900,21.446900,NaN,NaN,NaN,...,-2209110300000,2.387458e+06,8.652254e+06,NaN,NaN,N,Amount so small that it could not be measured,NaN,2016-05-01,2016-05-01 02:00:00
373,915,4648,Denmark,2016,NaN,57.417400,11.014700,7.380,1.800,NaN,...,-2209107600000,1.226151e+06,7.845915e+06,240,10.0,N,NaN,NaN,2016-08-13,2016-08-13 02:00:00


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OBJECTID        380 non-null    int64  
 1   HELCOM_ID       380 non-null    int64  
 2   Country         380 non-null    object 
 3   Year            380 non-null    int64  
 4   Spill_ID        378 non-null    object 
 5   Latitude        380 non-null    float64
 6   Longitude       380 non-null    float64
 7   Length__km      353 non-null    float64
 8   Width__km_      353 non-null    float64
 9   Area__km2_      359 non-null    float64
 10  Spill_cat       380 non-null    object 
 11  EstimVol_m3     365 non-null    float64
 12  Vol_Category    373 non-null    float64
 13  Polluter        375 non-null    object 
 14  Date            380 non-null    int64  
 15  Time_UTC        380 non-null    int64  
 16  x               380 non-null    float64
 17  y               380 non-null    flo

### save

In [5]:
data.to_csv("OIL.csv", index=False)

# Choose subset of 10 rows

### helpers

In [6]:
def convert_unix_time_to_standart_date(unix_time):
    return datetime.fromtimestamp(unix_time / 1000).date()

def convert_unix_time_to_standart_datetime(unix_time):
    return datetime.fromtimestamp(unix_time / 1000)

### convert time

In [7]:
data["Date_standard"] = data["Date"].apply(convert_unix_time_to_standart_date)
data["Datetime_standard"] = data["Date"].apply(convert_unix_time_to_standart_datetime)
data.head()

,OBJECTID,HELCOM_ID,Country,Year,Spill_ID,Latitude,Longitude,Length__km,Width__km_,Area__km2_,...,Time_UTC,x,y,Wind_speed,Wind_direc,FlightType,Remarks,Type_Substance,Date_standard,Datetime_standard
0,1003,4560,Sweden,2015,SE-83,63.519167,19.789833,1.70,0.09,0.153,...,-2209117500000,2.202994e+06,9.228699e+06,NaN,NaN,NaN,NaN,NaN,2015-11-06,2015-11-06 01:00:00
1,1004,4559,Sweden,2015,SE-60,57.733333,11.666667,5.50,0.20,1.100,...,-2209122000000,1.298727e+06,7.911507e+06,NaN,NaN,NaN,NaN,NaN,2015-08-07,2015-08-07 02:00:00
2,1005,4558,Sweden,2015,SE-81,58.616660,17.250000,0.70,0.30,0.210,...,-2209106100000,1.920261e+06,8.097990e+06,NaN,NaN,NaN,NaN,NaN,2015-10-30,2015-10-30 01:00:00
3,1006,4557,Sweden,2015,DK-23,56.183000,12.493500,1.10,0.10,0.110,...,-2209111680000,1.390770e+06,7.594932e+06,20,223.0,NaN,NaN,NaN,2015-05-29,2015-05-29 02:00:00
4,1007,4556,Sweden,2015,FI/SE-3,59.925000,19.427780,69.68,15.25,1.179,...,-2209116600000,2.162691e+06,8.383059e+06,NaN,NaN,NaN,NaN,NaN,2015-05-26,2015-05-26 02:00:00


### choose subset columns

In [42]:
data_oil_subset = data[["OBJECTID", "HELCOM_ID", "Spill_ID", "Latitude", "Longitude", "Length__km", "Width__km_", "Area__km2_", "EstimVol_m3",  "Date_standard", "Polluter"]]

### dropna

In [43]:
data_oil_subset = data_oil_subset.dropna(how="any")

### sort & filter

In [44]:
data_oil_10_record = data_oil_subset.sort_values(["Date_standard", "Area__km2_", "Length__km", "Width__km_"], ascending=False)
data_oil_10_record["Polluter"] = data_oil_10_record["Polluter"].str.upper()
data_oil_10_record = data_oil_10_record[data_oil_10_record["Polluter"] == "SHIP"]

### choose 10

In [45]:
data_oil_10_record = data_oil_10_record.head(10)
data_oil_10_record

,OBJECTID,HELCOM_ID,Spill_ID,Latitude,Longitude,Length__km,Width__km_,Area__km2_,EstimVol_m3,Date_standard,Polluter
273,6051,5787,GE/S-3,55.198333,14.065000,4.000,2.700,6.48000,0.427680,2023-06-20,SHIP
243,5697,5699,22UTSLÄPP0187,56.338611,17.351389,0.000,0.000,5.20000,0.031200,2022-10-23,SHIP
192,5576,5578,DK-22UTSLÄPP0181,57.672222,10.619167,0.000,0.000,0.04500,0.069440,2022-10-08,SHIP
198,5591,5593,FI-3,60.150700,26.522800,0.982,0.215,0.00000,0.050000,2022-08-02,SHIP
232,5669,5671,22UTSLÄPP0095,56.105000,15.483611,0.000,0.000,0.00100,0.000823,2022-06-28,SHIP
229,5663,5665,22UTSLÄPP0084,58.625833,16.799444,0.000,0.000,0.01500,0.001620,2022-06-14,SHIP
212,5622,5624,PL-16,55.172500,18.882700,0.300,0.030,0.00720,0.002000,2022-06-07,SHIP
225,5658,5660,22UTSLÄPP0071,55.418611,13.807222,0.000,0.000,0.01350,0.003812,2022-05-18,SHIP
224,5655,5657,22UTSLÄPP0067,59.440000,18.360278,0.000,0.000,0.00345,0.004289,2022-05-16,SHIP
223,5654,5656,22UTSLÄPP0068,55.635000,13.038056,0.000,0.000,0.00100,0.001358,2022-05-16,SHIP


### save

In [46]:
data_oil_10_record.to_csv("OIL_10.csv", index=False)